In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

/Users/dcolton/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/dcolton/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
df = pd.read_json('data.json')

In [10]:
df.body_length

0         3852
1         3499
2         2601
3        12347
4         2417
5          117
6           28
7          974
8         4388
9          974
10        4397
11        1679
12        7933
13        7138
14       10057
15        1615
16        1595
17         148
18        3387
19        6045
20        3530
21         452
22         742
23         810
24        1483
25        3254
26         870
27         475
28        1083
29        1901
         ...  
14307        0
14308     7444
14309     5736
14310     4999
14311      600
14312      831
14313        0
14314     6404
14315     1141
14316     3276
14317     6101
14318      448
14319     9574
14320      905
14321     2381
14322        0
14323     4809
14324    10108
14325      996
14326      541
14327     6007
14328     8149
14329    11930
14330      240
14331     4560
14332      667
14333     5812
14334      557
14335    10873
14336        0
Name: body_length, dtype: int64

In [3]:
a = df['acct_type'].unique()
uniq = list(a)

In [4]:
for i in uniq:
    print i, df[df['acct_type']==i].shape
print '***', 14337-12373, 'frauds ***'

fraudster_event (851, 44)
premium (12373, 44)
spammer_warn (144, 44)
fraudster (437, 44)
spammer_limited (218, 44)
spammer_noinvite (84, 44)
locked (54, 44)
tos_lock (77, 44)
tos_warn (91, 44)
fraudster_att (5, 44)
spammer_web (2, 44)
spammer (1, 44)
*** 1964 frauds ***


In [5]:
datecols = ['approx_payout_date', 'event_end', 'user_created']

convert = lambda x: datetime.date.fromtimestamp(x)
for i in datecols:
    df[i]=df[i].apply(pd.to_numeric)
    df[i]=df[i].apply(convert)

    
df['event_created'] = pd.to_datetime(df['event_created'],unit='s')
df['event_start'] = pd.to_datetime(df['event_start'],unit='s')
df['event_published'] = pd.to_datetime(df['event_published'],unit='s')

df['created_date'] = df['event_created'].dt.date
df['started_date'] = df['event_start'].dt.date
df['published_date'] = df['event_published'].dt.date

In [6]:
df['event_published'] = pd.to_datetime(df['event_published'],unit='s')

scatter_matrix(df, alpha=0.2, figsize=(16, 16), diagonal='kde')

def fraud_class(x):
    if x = "premium":
        return 0
    elif x = "spammer%":
        return 0
    else:
        return 1
df['fraud'] = df['acct_type'].apply(fraud_class)
df['fraud'].value_counts()

In [7]:
df['Fraud'] = ((df['acct_type'] == 'fraudster_event') | (df['acct_type'] == 'fraudster') | \
               (df['acct_type'] == 'locked') | (df['acct_type'] == 'tos_lock') | \
               (df['acct_type'] == 'tos_warn') | (df['acct_type'] == 'fraudster_att'))

In [12]:
df.shape

(14337, 49)

In [8]:
dfSAFE = df

In [9]:
g = df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}

{'bool': Index([u'Fraud'], dtype='object'),
 'datetime64[ns]': Index([u'event_created', u'event_published', u'event_start'], dtype='object'),
 'float64': Index([u'delivery_method', u'gts', u'has_header', u'org_facebook',
        u'org_twitter', u'sale_duration', u'venue_latitude',
        u'venue_longitude'],
       dtype='object'),
 'int64': Index([u'body_length', u'channels', u'fb_published', u'has_analytics',
        u'has_logo', u'name_length', u'num_order', u'num_payouts', u'object_id',
        u'sale_duration2', u'show_map', u'user_age', u'user_type'],
       dtype='object'),
 'object': Index([         u'acct_type', u'approx_payout_date',            u'country',
                  u'currency',        u'description',       u'email_domain',
                 u'event_end',             u'listed',               u'name',
                  u'org_desc',           u'org_name',         u'payee_name',
               u'payout_type',   u'previous_payouts',       u'ticket_types',
              u'

In [10]:
df['all_equal'] = ((df['event_start']==df['event_created']) & (df['event_start']==df['event_published']))

In [13]:
date_df = df[['Fraud','created_date','started_date','published_date']]
date_df['all_equal'] = ((date_df['created_date']==date_df['published_date']) & (date_df['created_date']==date_df['started_date']))

/Users/dcolton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
df['all_dates_equal'] = date_df['all_equal']

In [15]:
    keep_col = ['approx_payout_date', 'body_length', 'channels', 'country', 'currency', \
                'delivery_method', 'event_created', 'event_end', 'event_published', 'event_start', \
                'fb_published', 'gts', 'has_analytics', 'has_logo', 'listed', \
                'name_length', 'num_order', 'num_payouts', 'org_facebook', 'org_twitter', \
                'payout_type', 'sale_duration', 'sale_duration2', 'show_map', 'user_age', \
                'user_created', 'user_type', 'venue_country', 'venue_state']

In [17]:
df.shape

(14337, 50)

In [65]:
dfnum = df.drop(['venue_latitude', 'venue_longitude', 'venue_state', 'venue_name', 'country', 'currency', 'description'],inplace=False, axis=1)

In [66]:
dfnum = dfnum.drop(['email_domain', 'name', 'org_desc', 'payee_name','previous_payouts','ticket_types', 'venue_address'],inplace=False,axis=1)

In [67]:
dfnum = dfnum.drop(['org_name'], axis=1)

In [68]:
payout_df = pd.get_dummies(dfnum['payout_type'], drop_first=True)

In [70]:
dff = pd.concat([dfnum, payout_df],axis=1)

In [71]:
dff.fillna(0, inplace=True)

In [72]:
dff = dff.drop(['payout_type'], axis=1)

In [74]:
# y = df.pop('Fraud')
dff = dff.drop(['acct_type', 'event_created', 'event_published','event_start'], axis=1)

In [120]:
dfg = dff.drop(['approx_payout_date', 'event_end', 'listed', 'user_created', 'venue_country', 'created_date', 'started_date', 'published_date'], axis=1)

In [121]:
X = dfg.values

In [122]:
X

array([[3852, 5, 0.0, ..., False, 0, 0],
       [3499, 0, 1.0, ..., False, 0, 1],
       [2601, 8, 1.0, ..., False, 0, 1],
       ..., 
       [557, 13, 0.0, ..., False, 1, 0],
       [10873, 8, 0.0, ..., False, 1, 0],
       [0, 0, 0.0, ..., False, 1, 0]], dtype=object)

In [123]:
y.shape

(14337,)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [125]:
rf = RandomForestClassifier(max_features='sqrt', n_estimators=100, n_jobs=-1, random_state=1)
gb = GradientBoostingClassifier(learning_rate=.1, n_estimators=200, max_features='sqrt', max_depth=5, random_state=1)
ab = AdaBoostClassifier(n_estimators=200)

In [126]:
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)
ab.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=None)

In [131]:
print 'RF:', rf.score(X_test,y_test)
print 'GB:', gb.score(X_test,y_test)
print 'AB:', ab.score(X_test,y_test)

RF: 0.966062296606
GB: 0.968386796839
AB: 0.955834495583


In [137]:
models = [rf, gb, ab]

for i in models:
    print '{} Accuracy: {}'.format(1, i.score(X_test, y_test))
    print '{} Precision: {}'.format(1, precision_score(y_test, i.predict(X_test)))
    print '{} Recall: {}'.format(1, recall_score(y_test, i.predict(X_test)))

1 Accuracy: 0.966062296606
1 Precision: 0.942857142857
1 Recall: 0.723684210526
1 Accuracy: 0.968386796839
1 Precision: 0.921052631579
1 Recall: 0.767543859649
1 Accuracy: 0.955834495583
1 Precision: 0.861413043478
1 Recall: 0.695175438596


In [150]:
def balancer(df, label, margin = 0.4, size = 500):
    yes = df[df[label] == 1]
    no = df[df[label] == 0]
    return yes.sample(int(size * margin),axis =0).append(no.sample(size - int(size * margin),axis =0))

In [148]:
dfg['Fraud'] = ((df['acct_type'] == 'fraudster_event') | (df['acct_type'] == 'fraudster') | \
               (df['acct_type'] == 'locked') | (df['acct_type'] == 'tos_lock') | \
               (df['acct_type'] == 'tos_warn') | (df['acct_type'] == 'fraudster_att'))